In [ ]:
!pip install get-all-tickers

In [ ]:
"""
https://pypi.org/project/yfinance/
https://algotrading101.com/learn/yfinance-guide/
"""

import csv 
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import datetime
from get_all_tickers import get_tickers as gt
import re

def options_chain(symbol): 
    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)
        
    # Account for error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary columns
    #options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])
    options = options.drop(columns = ['currency', 'change', 'percentChange', 'lastPrice'])
    options['Timestamp'] = dt.datetime.now()
    options['Ticker'] = symbol
    
    return options

#Import all relevant tickers
tickers = gt.get_tickers()
unique_tickers = list(set(tickers))
options_dataframe = pd.DataFrame()

#Output options data on all tickers
for idx,x in enumerate(unique_tickers, start = 0):
    try:     
        options_dataframe = options_dataframe.append(options_chain(x), ignore_index=True)       
    except IndexError:
        continue
    except KeyError:
        pass
    except AttributeError:
        pass
    #if idx == 5:  #run first 10 indexes for test run
        #break      #run first 10 indexes for test run

#Convert options contract symbol to ticker

#export data
options_dataframe.to_csv("Options_data.csv", sep=',', na_rep='empty',index_label=False, index=False)
